In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import tree
import random
import collections
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
resp = pd.read_csv("young-people-survey/responses.csv")
podcasts=pd.read_csv("young-people-survey/df_popular_podcasts.csv")
word_exp=re.compile("[^\x00-\x7F]+")
non_eng_pod=[index for index,value in enumerate(list(podcasts["Name"].to_dict().values())) if len(re.findall(word_exp,value))!=0]
podcasts=podcasts.drop(non_eng_pod).drop_duplicates("Name", keep="first")

movies=pd.read_csv("young-people-survey/movie_metadata.csv")
movies=movies[movies["imdb_score"].astype(float)>float(5.0)]
movies=movies.dropna().drop_duplicates("movie_imdb_link", keep="first")

music=pd.read_csv("young-people-survey/SpotifyFeatures.csv")
music=music.drop_duplicates("track_id")
music=music[music["popularity"].astype(float)>float(50)]

genre_IDs=[['1311', 'News & Politics'], ['26', 'Podcasts'], ['1479', 'Social Sciences'], ['1315', 'Science & Medicine'], ['1324', 'Society & Culture'], ['1302', 'Personal Journals'], ['1469', 'Language Courses'], ['1304', 'Education'], ['1320', 'Places & Travel'], ['1416', 'Higher Education'], ['1465', 'Professional'], ['1316', 'Sports & Recreation'], ['1303', 'Comedy'], ['1305', 'Kids & Family'], ['1439', 'Christianity'], ['1314', 'Religion & Spirituality'], ['1444', 'Spirituality'], ['1309', 'TV & Film'], ['1462', 'History'], ['1310', 'Music'], ['1478', 'Medicine'], ['1321', 'Business'], ['1412', 'Investing'], ['1420', 'Self-Help'], ['1307', 'Health'], ['1481', 'Alternative Health'], ['1417', 'Fitness & Nutrition'], ['1467', 'Amateur'], ['1480', 'Software How-To'], ['1318', 'Technology'], ['1448', 'Tech News'], ['1456', 'Outdoor'], ['1477', 'Natural Sciences'], ['1301', 'Arts'], ['1454', 'Automotive'], ['1323', 'Games & Hobbies'], ['1438', 'Buddhism'], ['1443', 'Philosophy'], ['1401', 'Literature'], ['1402', 'Design'], ['1410', 'Careers'], ['1470', 'Training'], ['1413', 'Management & Marketing'], ['1306', 'Food'], ['1406', 'Visual Arts'], ['1446', 'Gadgets'], ['1468', 'Educational Technology'], ['1405', 'Performing Arts'], ['1460', 'Hobbies'], ['1471', 'Business News'], ['1404', 'Video Games'], ['1450', 'Podcasting'], ['1473', 'National'], ['1325', 'Government & Organizations'], ['1461', 'Other Games'], ['1466', 'College & High School'], ['1459', 'Fashion & Beauty'], ['1476', 'Non-Profit'], ['1415', 'K-12'], ['1455', 'Aviation'], ['1464', 'Other'], ['1421', 'Sexuality'], ['1472', 'Shopping'], ['1475', 'Local'], ['1441', 'Judaism'], ['1440', 'Islam'], ['1474', 'Regional'], ['1463', 'Hinduism']]

In [167]:
movies.genres

0           Action|Adventure|Fantasy|Sci-Fi
1                  Action|Adventure|Fantasy
2                 Action|Adventure|Thriller
3                           Action|Thriller
5                   Action|Adventure|Sci-Fi
                       ...                 
5026                    Drama|Music|Romance
5027                                  Drama
5033                  Drama|Sci-Fi|Thriller
5035    Action|Crime|Drama|Romance|Thriller
5042                            Documentary
Name: genres, Length: 3336, dtype: object

user says: ['Romance', 'Documentary'] 
i need a dataframe that has rows that contain either romance or documentary, and they can have other genres in their string too 

ideally a new dataframe that just has rows that match these conditions 
need plot_keywords for all movies that match these conditions 

In [4]:
# Better organize the id vs name of genre
dict_pod={int(id[0]):id[1] for id in genre_IDs }

# {Genre ID:NAMES OF PODCAST}
genre_to_podcast={key:[] for key in dict_pod }
for item in podcasts.index:
    array=podcasts["Genre IDs"][item][1:-1].split(',')
    array=[int(array[i].replace('\'', '').strip('\'')) for i in range(len(array))]
    for j in range(len(array)):
        genre_to_podcast[array[j]].append((podcasts["Name"][item], podcasts["Description"][item], podcasts["Podcast URL"][item]))
final_dict={list(dict_pod.values())[p]:list(genre_to_podcast.values())[p] for p in range(len(genre_to_podcast.keys()))}
final_dict = dict((k.lower(), v) for k, v in final_dict.items())

#MAKE MOVIE DICTIONARY
genre_to_movie={}
for i in range( len(movies)):
    line=str(movies.iloc[i]["Genre"])
    for genre in line.split("|"):
        genre=genre.lower()
        if genre in genre_to_movie:
            genre_to_movie[genre].append(movies.iloc[i]["Title"])
        else:
            genre_to_movie[genre]=[movies.iloc[i]["Title"]]

#MAKE MUSIC DICTIONARY
genre_to_music={}
for i in range(len(music)):
    genre=music.iloc[i]['genre']
    genre=genre.lower()
    if genre in genre_to_music:
            genre_to_music[genre].append((music.iloc[i]['track_id'], music.iloc[i]['track_name'],  music.iloc[i]['genre'],  music.iloc[i]['artist_name']))
    else:
            genre_to_music[genre]=[(music.iloc[i]['track_id'], music.iloc[i]['track_name'],  music.iloc[i]['genre'],  music.iloc[i]['artist_name'])]


KeyError: 'Genre'

In [5]:
song=recs(genre_to_music, ["Folk"])

NameError: name 'recs' is not defined

In [ ]:
song=[{
            'title': item[0][1],
            'url': "https://open.spotify.com/embed/track/"+item[0][0],
            'artist': item[0][3], 
            'genre': item[0][2]
            } for item in song]

In [ ]:
podcast=recs(final_dict,["News & Politics"])

In [ ]:
podcast=[{
            'title': item[0][0],
            'description': item[0][1],
            'url': item[0][2]
            } for item in podcast]

In [5]:
def cosine_sim(corpus):
    vectorizer=TfidfVectorizer(stop_words="english", min_df=1)
    tfidf = vectorizer.fit_transform(corpus)
    similarity = tfidf * tfidf.T
    return similarity.toarray()

In [6]:
cosine_sim(["I'd like an apple",  "An apple a day keeps the doctor away"])

array([[1.        , 0.19431434],
       [0.19431434, 1.        ]])

{'Sci-Fi', 'Comedy', 'Mystery', 'Biography', 'Musical', 'Thriller', 'Family', 'Romance', 'Adventure', 'History', 'Western', 'Drama', 'Music', 'Crime', 'War', 'Animation', 'Action', 'Horror', 'Documentary', 'Film-Noir', 'Fantasy', 'Sport'}


In [196]:
genre_to_movie={}
for i in range(len(movies)):
    line=str(movies.iloc[i]["genres"])
    for genre in line.split("|"):
        genre=genre.lower()
        if genre in genre_to_movie:
            genre_to_movie[genre].append(movies.iloc[i]["movie_imdb_link"])
        else: 
            genre_to_movie[genre]=[movies.iloc[i]["movie_imdb_link"]]


In [192]:
genre_to_movie={}

for i in range(len(movies)):
    line=str(movies.iloc[i]["genres"])
    for genre in line.split("|"):
        genre=genre.lower()
        if genre in genre_to_movie:
            genre_to_movie[genre].append(movies.iloc[i]["movie_title"])
        else: 
            genre_to_movie[genre]=[movies.iloc[i]["movie_title"]]


In [194]:
genre_to_movie['musical']

['Tangled\xa0',
 'Frozen\xa0',
 'Happy Feet 2\xa0',
 'Rio 2\xa0',
 'The Princess and the Frog\xa0',
 'The Hunchback of Notre Dame\xa0',
 'Mulan\xa0',
 'Rio\xa0',
 'Enchanted\xa0',
 'Nine\xa0',
 'The Lion King\xa0',
 'Hairspray\xa0',
 'Dreamgirls\xa0',
 'Rock of Ages\xa0',
 'Les Misérables\xa0',
 'Annie\xa0',
 'The Prince of Egypt\xa0',
 'Pocahontas\xa0',
 'Evita\xa0',
 'The Phantom of the Opera\xa0',
 'Muppets Most Wanted\xa0',
 'Burlesque\xa0',
 'Anastasia\xa0',
 'Moulin Rouge!\xa0',
 'Mamma Mia!\xa0',
 'Into the Woods\xa0',
 'The Muppets\xa0',
 'Across the Universe\xa0',
 'The Producers\xa0',
 'Pinocchio\xa0',
 'Jersey Boys\xa0',
 'The Doors\xa0',
 'Rent\xa0',
 'The Magic Sword: Quest for Camelot\xa0',
 "Cats Don't Dance\xa0",
 'Chicago\xa0',
 'Corpse Bride\xa0',
 'Little Shop of Horrors\xa0',
 'Winnie the Pooh\xa0',
 'Aladdin\xa0',
 'The Rugrats Movie\xa0',
 'Rugrats Go Wild\xa0',
 'The Adventures of Pinocchio\xa0',
 'The Wiz\xa0',
 'Beyond the Sea\xa0',
 'Darling Lili\xa0',
 'South

In [200]:
def filter_mov(query):
    mod_movies=pd.DataFrame()
    for value in query:
        mod_movies=mod_movies.append(movies[movies["movie_imdb_link"].isin(genre_to_movie[value])])
    return mod_movies.drop_duplicates("movie_imdb_link", keep="first")

filter_mov(['musical']).movie_title.to_dict().values()



dict_values(['Tangled\xa0', 'Frozen\xa0', 'Happy Feet 2\xa0', 'Rio 2\xa0', 'The Princess and the Frog\xa0', 'The Hunchback of Notre Dame\xa0', 'Mulan\xa0', 'Rio\xa0', 'Enchanted\xa0', 'Nine\xa0', 'The Lion King\xa0', 'Hairspray\xa0', 'Dreamgirls\xa0', 'Rock of Ages\xa0', 'Les Misérables\xa0', 'Annie\xa0', 'The Prince of Egypt\xa0', 'Pocahontas\xa0', 'Evita\xa0', 'The Phantom of the Opera\xa0', 'Muppets Most Wanted\xa0', 'Burlesque\xa0', 'Anastasia\xa0', 'Moulin Rouge!\xa0', 'Mamma Mia!\xa0', 'Into the Woods\xa0', 'The Muppets\xa0', 'Across the Universe\xa0', 'The Producers\xa0', 'Pinocchio\xa0', 'Jersey Boys\xa0', 'The Doors\xa0', 'Rent\xa0', 'The Magic Sword: Quest for Camelot\xa0', "Cats Don't Dance\xa0", 'Chicago\xa0', 'Corpse Bride\xa0', 'Little Shop of Horrors\xa0', 'Winnie the Pooh\xa0', 'Aladdin\xa0', 'The Rugrats Movie\xa0', 'Rugrats Go Wild\xa0', 'The Adventures of Pinocchio\xa0', 'The Wiz\xa0', 'Beyond the Sea\xa0', 'Darling Lili\xa0', 'South Park: Bigger Longer & Uncut\xa0',

In [214]:
music.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
135,R&B,Mary J. Blige,Be Without You - Kendu Mix,2YegxR5As7BeQuVp2U6pek,65,0.0830,0.724,246333,0.689,0.000000,D,0.3040,-5.922,Minor,0.1350,146.496,4-Apr,0.6930
136,R&B,Rihanna,Desperado,6KFaHC9G178beAp7P0Vi5S,63,0.3230,0.685,186467,0.610,0.000000,C,0.1020,-5.221,Minor,0.0439,94.384,4-Mar,0.3230
137,R&B,Yung Bleu,Ice On My Baby (feat. Kevin Gates) - Remix,6muW8cSjJ3rusKJ0vH5olw,62,0.0675,0.762,199520,0.520,0.000004,F,0.1140,-5.237,Minor,0.0959,75.047,4-Apr,0.0862
138,R&B,Surfaces,Heaven Falls / Fall on Me,7yHqOZfsXYlicyoMt62yC6,61,0.3600,0.563,240597,0.366,0.002430,B,0.0955,-6.896,Minor,0.1210,85.352,4-Apr,0.7680
139,R&B,Olivia O'Brien,Love Myself,4XzgjxGKqULifVf7mnDIQK,68,0.5960,0.653,213947,0.621,0.000000,B,0.0811,-5.721,Minor,0.0409,100.006,4-Apr,0.4660


In [243]:
def filter_mov(query):
    mod_movies=pd.DataFrame()
    for value in query:
        mod_movies=mod_movies.append(movies[movies["movie_imdb_link"].isin(genre_to_movie[value])])
    return mod_movies.drop_duplicates("movie_imdb_link", keep="first")

def cosine_sim(corpus):
    vectorizer=TfidfVectorizer(stop_words="english", min_df=1)
    tfidf = vectorizer.fit_transform(corpus)
    similarity = tfidf * tfidf.T
    return similarity.toarray()

def get_max_val_podcast(np_array): 
    index_max_val=np.argmax(np_array)
    output=(podcasts.iloc[index_max_val][:]["Name"], podcasts.iloc[index_max_val][:]["Description"], podcasts.iloc[index_max_val][:]["Podcast URL"], np_array[index_max_val])
    if np_array[index_max_val]==0:
        return 
    else: 
        np_array[index_max_val]=0
    return output

def get_max_val_movie(np_array, movies_filtered):
    index_max_val=np.argmax(np_array)
    output=(movies_filtered.iloc[index_max_val][:]["movie_title"], movies_filtered.iloc[index_max_val][:]["title_year"], movies_filtered.iloc[index_max_val][:]["num_voted_users"], movies_filtered.iloc[index_max_val][:]["movie_imdb_link"], movies_filtered.iloc[index_max_val][:]["imdb_score"],np_array[index_max_val],  movies_filtered.iloc[index_max_val][:]["plot_keywords"], movies_filtered.iloc[index_max_val][:]["genres"])
    if np_array[index_max_val]==0:
        return 
    else: 
        np_array[index_max_val]=0
    return output

def podcast_recs(query):
    descriptions=list(podcasts['Description'])
    corpus=[query]+descriptions
    matrix=cosine_sim(corpus)
    matrix_slice=matrix[:][0][1:]
    result=[]
    for x in range(3):
        result.append(get_max_val_podcast(matrix_slice))
    while None in result:
        result.remove(None)
    return result

def movie_filter(genres, query):
    overlap=[]
    for genre_exp in genres:
        for genre in query:
            if genre in genre_exp:
                overlap.append(genre_exp)
    return overlap

def movie_recs(query, genres=None):
    if genres is None:
        movies_filtered = movies
    else:
        genres=[genre.lower() for genre in genres]
        movies_filtered=filter_mov(genres)
        #print(movies_filtered.movie_title)
        #print(movies_filtered.loc[movies_filtered['movie_title']=='Maleficent'])
        movies_filtered = movies_filtered.sample(frac=1).reset_index(drop=True)
        if query == "" :
            result=[]
            for i in range(3): 
                output=(movies_filtered.iloc[i][:]["movie_title"], movies_filtered.iloc[i][:]["title_year"], movies_filtered.iloc[i][:]["num_voted_users"], movies_filtered.iloc[i][:]["movie_imdb_link"], movies_filtered.iloc[i][:]["imdb_score"], "N/A",  movies_filtered.iloc[i][:]["plot_keywords"], movies_filtered.iloc[i][:]["genres"])
                result.append(output)
            print("hi")
            return result 
        
    plot_keywords=list(movies_filtered['plot_keywords'])
    #print(plot_keywords)
    for i in range(len(plot_keywords)):
        plot_keywords[i]=plot_keywords[i].replace('|', " ")
    corpus=[query]+plot_keywords
    matrix=cosine_sim(corpus)
    matrix_slice=matrix[:][0][1:]
    result=[]
    for x in range(3):
        movie=get_max_val_movie(matrix_slice, movies_filtered)
        result.append(movie)
    while None in result:
        result.remove(None)
    return result

In [260]:
movie=movie_recs("", ['Romance'])

hi


In [261]:
movie

[('Out of Africa\xa0',
  1985.0,
  52339,
  'http://www.imdb.com/title/tt0089755/?ref_=fn_tt_tt_1',
  7.2,
  'N/A',
  'africa|hunter|love|marriage|plantation',
  'Biography|Drama|Romance'),
 ('High Fidelity\xa0',
  2000.0,
  143137,
  'http://www.imdb.com/title/tt0146882/?ref_=fn_tt_tt_1',
  7.6,
  'N/A',
  'employer employee relationship|record store|store|thirty something|vinyl',
  'Comedy|Drama|Music|Romance'),
 ('The Wood\xa0',
  1999.0,
  6585,
  'http://www.imdb.com/title/tt0161100/?ref_=fn_tt_tt_1',
  6.9,
  'N/A',
  'flashback|friendship|public nudity|shower|wedding',
  'Comedy|Drama|Romance')]

In [236]:
music.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
135,R&B,Mary J. Blige,Be Without You - Kendu Mix,2YegxR5As7BeQuVp2U6pek,65,0.0830,0.724,246333,0.689,0.000000,D,0.3040,-5.922,Minor,0.1350,146.496,4-Apr,0.6930
136,R&B,Rihanna,Desperado,6KFaHC9G178beAp7P0Vi5S,63,0.3230,0.685,186467,0.610,0.000000,C,0.1020,-5.221,Minor,0.0439,94.384,4-Mar,0.3230
137,R&B,Yung Bleu,Ice On My Baby (feat. Kevin Gates) - Remix,6muW8cSjJ3rusKJ0vH5olw,62,0.0675,0.762,199520,0.520,0.000004,F,0.1140,-5.237,Minor,0.0959,75.047,4-Apr,0.0862
138,R&B,Surfaces,Heaven Falls / Fall on Me,7yHqOZfsXYlicyoMt62yC6,61,0.3600,0.563,240597,0.366,0.002430,B,0.0955,-6.896,Minor,0.1210,85.352,4-Apr,0.7680
139,R&B,Olivia O'Brien,Love Myself,4XzgjxGKqULifVf7mnDIQK,68,0.5960,0.653,213947,0.621,0.000000,B,0.0811,-5.721,Minor,0.0409,100.006,4-Apr,0.4660


In [239]:
movie=[{
            'title': item[0],
            'year': item[1],
            'link': item[3], 
            'num_revs': item[2],
            'rating': item[4],
            'score': item[5], 
            'words': item[6].replace("|", ", "), 
            'genres': item[7].replace("|", ", ")
            } for item in movie]

In [238]:
movie

[]

In [ ]:
for podcast in podcasts:
    print(podcast)

In [ ]:
podcast=[{
            'title': item[0],
            'description': item[1],
            'url': item[2], 
            'score': item[3]
            } for item in podcast]

In [ ]:
podcast=podcast_recs("alsfkn")

In [ ]:
podcast[0]==()

In [ ]:
def recs(genre_dict, genre_query):
    """Returns a list of recommendations based on interests user clicked in form
    Params: {query: list of genre names, genre_dict: dictionary that maps genre to titles, corr_dict: maps personality questions to answers given in survey}
    Returns: list of tuples containing titles and scores
    """

    genre_query=[genre.lower() for genre in genre_query]

    counter={}
    for cat in genre_query:
        for film in genre_dict[cat]:
            if film in counter:
                counter[film]+=1
            else:
                counter[film]=1
    results = list(counter.items())
    random.shuffle(results)  
    results.sort(key=lambda x: x[1], reverse=True)

    return results